In [290]:
from Module.Record import record
from Module.train import finetune
from Module.Results import correctly_identify, could_not_identify
from Module.env import inference

In [256]:
#Imports
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from sklearn.model_selection import train_test_split

from pyannote.audio import Inference
# from scipy.spatial.distance import cdist
# instantiate pretrained model
from pyannote.audio import Model

import pyaudio
import wave
import json

In [257]:
#dict for all the recorded names
names={}

#data folder path
folder_path = "E:/New-Codes/Repositories/IdentifyMe-AI/VoiceData/data" 

In [258]:

# Define parameters
FORMAT = pyaudio.paInt16   # Format of audio
CHANNELS = 1               # Number of channels (1 for mono, 2 for stereo)
RATE = 44100               # Sample rate
CHUNK = 1024               # Buffer size
RECORD_SECONDS = 6        # Duration of recording



# To record the audio
def record(audio):
    # Create an interface to PortAudio
    

    stream = audio.open(format=FORMAT, channels=CHANNELS,
                        rate=RATE, input=True,
                        frames_per_buffer=CHUNK)

    print("Recording...")

    frames = []

    for _ in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)

    print("Finished recording.")

    # Stop and close the stream
    stream.stop_stream()
    stream.close()
    audio.terminate()

    # Save the recorded data as a WAV file
    with wave.open("recorded.wav" , 'wb') as wf:
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(audio.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))

    print("Recorded audio saved as recorded.wav ")
    return frames



In [259]:
def finetune(X_new,y_new):

    X_old=np.load("X_embeddings.npy")
    y_old=np.load("y_labels.npy")


    X_finetune = np.concatenate([X_old, X_new.reshape(1,-1)], axis=0)
    y_finetune = np.concatenate([y_old, y_new.reshape(1)], axis=0)

    model=load_model("updated_model.keras")

    history = model.fit(X_finetune, y_finetune, epochs=10, batch_size=8, validation_split=0.2)

    model.save("updated_model.keras")



In [260]:
#if couldnt identify
def could_not_identify(embeddings,audio,frames,names):
    

    que=input(f"Are you a new Speaker?   (y/n)")
    is_new= False if (que=="n" or que=="n") else True
    new_name="Prashasst"
    new_label=0

    if not is_new:
        print(names)
        new_label=int(input("Enter your id : "))
        new_name=names[new_label]
        np.save("X_new_embeddings.npy",embeddings)
        np.save("y_new_labels.npy",new_label)
        finetune(embeddings,np.array(new_label))
    else:
        new_name=input("Enter your correct name: ")
        new_label= list(names.keys())[-1]+1
        new_name_folder_path=f"{folder_path}/{new_label}_{new_name}"
        os.makedirs(new_name_folder_path)

        print(" Retrain the model from scratch ")
    

    # new_name_folder_path=f"{folder_path}/{new_label}_{new_name}"

    # if new_name in names:
    #     print("It looks like your name is already saved.")
    #     new_label=names[new_name]

    

    new_name_folder_path=f"{folder_path}/{new_label}_{new_name}"

    # Saving the recorded data as a WAV file
    with wave.open(f"{new_name_folder_path}/{np.random.randint(100)}_{new_name}.wav" , 'wb') as wf:
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(audio.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))

    print(f"Recorded audio saved in {new_label}_{new_name} in voiceData/data folder")


In [261]:
#if identifued correctly
def correctly_identify(embeddings,the_label,audio,frames,names):

    
    
    que=input(f"Were you identifed correctly? (y/n)")
    correct= True if (que=="y" or que=="Y") else False

    if not correct:
        print(names)
        the_label=int(input("Enter your correct id  or -1 if not here: "))
        if the_label==-1:return
        the_name=names[the_label]


    the_name=names[the_label]

    np.save("X_new_embeddings.npy",embeddings)
    np.save("y_new_labels.npy",the_label)

    finetune(embeddings,np.array(the_label))

    

    the_name_folder_path=f"{folder_path}/{the_label}_{the_name}"

   

    # Saving the recorded data as a WAV file
    with wave.open(f"{the_name_folder_path}/{np.random.randint(100)}_{the_name}.wav" , 'wb') as wf:
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(audio.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))

    print(f"Recorded audio saved in {the_label}_{the_name} in voiceData/data folder")


In [262]:
# new_label= list(names.keys())[-1]+1
# print(new_label)

In [263]:
# for extracting vector embeddings from the audio files 

model = Model.from_pretrained("pyannote/wespeaker-voxceleb-resnet34-LM",cache_dir=r"C:\Users\prash\.cache\huggingface\hub\models--pyannote--wespeaker-voxceleb-resnet34-LM")
inference = Inference(model, window="whole")

In [264]:
#making the X and y for model


final=[]
y_label=[]
for labeled_name in os.listdir(folder_path):
    label=int(labeled_name[0])
    name =labeled_name[2:]

    if label not in names:names[label]=name
    
    
    file_path=f"{folder_path}/{labeled_name}/"
    # print(file_path)
    for wav_file in os.listdir(file_path):
        # print(f"{file_path}{wav_file}")
        final.append(inference(f"{file_path}{wav_file}"))
        y_label.append(label)

X=np.array(final)
y=np.array(y_label)

# Saving the labels to a json
with open('names_identified.json', 'w') as f:
    json.dump(names, f)


In [265]:
names

{0: 'Prashasst',
 1: 'Sarthak',
 2: 'Satendra',
 3: 'Shraddha',
 4: 'Shreyansh',
 5: 'Sourabh',
 6: 'shivam_sharma',
 7: 'Rupesh'}

In [266]:
print(f"X shape: {X.shape} y shape: {y.shape}")

X shape: (69, 256) y shape: (69,)


In [267]:
np.save("X_embeddings.npy",X)
np.save("y_labels.npy",y)

In [268]:
#spliting into training and validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=49)

In [269]:


# Define the model
model = tf.keras.Sequential([
    InputLayer(input_shape=(256,), name='input_layer'),  
    tf.keras.layers.Reshape((1, 256, 1)),  

    # Convolutional layers
    Conv2D(32, (1, 3), activation='relu', padding='same'),  # 1x3 convolution for embedding
    MaxPooling2D(pool_size=(1, 2)),  # Downsample

    Conv2D(64, (1, 3), activation='relu', padding='same'),  # 1x3 convolution
    MaxPooling2D(pool_size=(1, 2)),  # Downsample

    Conv2D(128, (1, 3), activation='relu', padding='same'),  # 1x3 convolution
    MaxPooling2D(pool_size=(1, 2)),  # Downsample

    Flatten(),  # Flatten the output for dense layers

    # Fully connected layers
    Dense(256, activation='relu'),
    Dropout(0.5),  # Regularization to prevent overfitting
    Dense(128, activation='relu'),
    Dropout(0.5),  # Regularization
    Dense(len(names), activation='softmax')  # Output layer for classification
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ reshape_8 (Reshape)             │ (None, 1, 256, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_24 (Conv2D)              │ (None, 1, 256, 32)     │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_24 (MaxPooling2D) │ (None, 1, 128, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_25 (Conv2D)              │ (None, 1, 128, 64)     │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_25 (MaxPooling2D) │ (None, 1, 64, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_26 (Conv2D)              │ (None, 1, 64, 128)     │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_26 (MaxPooling2D) │ (None, 1, 32, 128)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_8 (Flatten)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 256)            │     1,048,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_17 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 8)              │         1,032 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,113,800 (4.25 MB)

 Trainable params: 1,113,800 (4.25 MB)

 Non-trainable params: 0 (0.00 B)

In [270]:
history = model.fit(X_train, y_train,
                    epochs=200,            # Adjust the number of epochs as needed
                    batch_size=32,        # Adjust based on your data size
                    validation_data=(X_val, y_val))

Epoch 1/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 204ms/step - accuracy: 0.1280 - loss: 2.0864 - val_accuracy: 0.5714 - val_loss: 1.9543
Epoch 2/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.3293 - loss: 2.0219 - val_accuracy: 0.5714 - val_loss: 1.7679
Epoch 3/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.3616 - loss: 1.9342 - val_accuracy: 0.5714 - val_loss: 1.6023
Epoch 4/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.3401 - loss: 1.8847 - val_accuracy: 0.5714 - val_loss: 1.5156
Epoch 5/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.2984 - loss: 1.9584 - val_accuracy: 0.5714 - val_loss: 1.5896
Epoch 6/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.3091 - loss: 1.9523 - val_accuracy: 0.5714 - val_loss: 1.6608
Epoch 7/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.3306 - loss: 1.8246 - val_accuracy: 0.5714 - val_loss: 1.6664
Epoch 8/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.3935 - loss: 1.8219 - val_accuracy: 0.5714 - val_loss

In [271]:
model.save("updated_model.keras")

In [287]:
with open("names_identified.json", 'r') as f:
    loaded_json = json.load(f)
identified_names={int(k):v for k,v in loaded_json.items()}

model=load_model("updated_model.keras")

In [288]:
#testing the model




audio = pyaudio.PyAudio()
frames=record(audio=audio)

test=inference(r"E:\New-Codes\Repositories\IdentifyMe-AI\recorded.wav")
predection=model.predict(test.reshape(1,-1))

threshold = 0.6  

# Checking if the highest probability exceeds the threshold
result = np.argmax(predection) if  np.max(predection) > threshold else -1


identified= True if result!=-1 else False


Recording...
Finished recording.
Recorded audio saved as recorded.wav 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step


In [289]:
if result==-1:print("Could not identify")
else:print(f"Person identified as {names[result]}")

Could not identify


In [275]:
# print( f"old {X.shape} new {test.reshape(1,-1).shape}")
# print( f"old {y.shape} new {result.reshape(1).shape}")

In [286]:
#if could not identify

if not identified: could_not_identify(embeddings=test,audio=audio,frames=frames,names=identified_names)
else:correctly_identify(embeddings=test,the_label=result,audio=audio,frames=frames,names=identified_names)

Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 58ms/step - accuracy: 1.0000 - loss: 0.0010 - val_accuracy: 1.0000 - val_loss: 0.0330
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9573 - loss: 0.0499 - val_accuracy: 1.0000 - val_loss: 0.0782
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 1.0000 - loss: 0.0044 - val_accuracy: 0.9286 - val_loss: 0.0932
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.9379 - loss: 0.3032 - val_accuracy: 1.0000 - val_loss: 0.0193
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.9729 - loss: 0.0438 - val_accuracy: 1.0000 - val_loss: 0.0307
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 1.0000 - loss: 0.0080 - val_accuracy: 1.0000 - val_loss: 0.0308
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 1.0000 - loss: 0.0131 - val_accuracy: 1.0000 - val_loss: 0.0227
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 1.0000 - loss: 0.0062 - val_accuracy: 1.0000 - val_loss: 0.0198
